# 문제: 사용자에게 영화 또는 프로그램 추천

기본 소스:
- [Implementing a Recommender System with SageMaker, MXNet, and Gluon](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_applying_machine_learning/gluon_recommender_system/gluon_recommender_system.ipynb)
- [An Introduction to Factorization Machines with MNIST](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/factorization_machines_mnist/factorization_machines_mnist.ipynb)
- [Extending Amazon SageMaker Factorization Machines Algorithm to Predict Top X Recommendations](https://aws.amazon.com/blogs/machine-learning/extending-amazon-sagemaker-factorization-machines-algorithm-to-predict-top-x-recommendations/)

## 비즈니스 시나리오 소개

여러분은 사용자에게 온디맨드 비디오 스트리밍 서비스를 제공하는 스타트업에서 일합니다. 이 회사는 시청 기록에 따라 사용자에게 영화/프로그램 추천을 제공하려고 합니다.

여러분은 기계 학습을 활용하여 사용자 웹 사이트에서 사용할 추천 엔진을 만들어 이 문제의 일부를 해결해야 합니다. 여러분에게는 사용자 기본 설정 기록 및 시청한 영화의 데이터 집합에 액세스할 수 있는 권한이 부여됩니다. 시청할 영화/프로그램을 추천하도록 기계 학습 모형을 훈련하는 데 이를 사용할 수 있습니다.

## 이 데이터 집합 소개  
Amazon Customer Reviews Dataset은 1995년부터 2015년까지 Amazon.com 마켓플레이스의 다양한 제품에 대한 리뷰 모음입니다. 고객 리뷰는 Amazon에서 가장 중요한 데이터 유형 중 하나입니다. 리뷰 수집 및 게시는 회사 초기부터 Amazon 문화의 일부였으며 단언컨대 혁신의 중요한 원천입니다. 이 데이터 집합에 대한 자세한 내용은 [Amazon Customer Reviews Dataset](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)를 참조하십시오.

이 연습은 비디오 리뷰에 초점을 맞춥니다. 이 비디오 데이터 집합에는 160,000개의 디지털 비디오에 대해 2백만 명 이상의 Amazon 고객이 매긴 별 1개~5개의 평점이 포함되어 있습니다.

### 특성

**데이터 열**

- `marketplace`: 2자 국가 코드(이 예제에서는 모두 "US")
- `customer_id`: 단일 작성자가 작성한 리뷰를 집계하는 데 사용할 수 있는 랜덤 식별자
- `review_id`: 리뷰의 고유 ID
- `product_id`: ASIN(아마존 표준 식별 번호. http://www.amazon.com/dp/&lt;ASIN\>는 제품 세부 정보 페이지로 연결됩니다.
- `product_parent`: 해당 ASIN의 상위 상품입니다. 여러 개의 ASIN(동일한 상품의 색상 또는 형식 변형)이 단일 상위 상품으로 겹쳐서 표시될 수 있습니다.
- `product_title`: 상품의 타이틀 설명
- `product_category`: 리뷰 그룹화에 사용할 수 있는 광범위한 상품 카테고리(이 예제에서는 디지털 비디오)
- `star_rating`: 상품 평점(별 1개~5개)
- `helpful_votes`: 리뷰가 받은 “도움 돼요” 투표 수
- `total_votes`: 리뷰가 받은 “도움 돼요” 총 투표 수
- `vine`: 리뷰는 Vine 프로그램의 일부로 작성되었나요?
- `verified_purchase`: 확인된 구매의 리뷰인가요?
- `review_headline`: 리뷰 자체의 제목
- `review_body`: 리뷰 내용
- `review_date`: 리뷰가 작성된 날짜


**데이터 형식**
- 따옴표 또는 이스케이프 문자 없이 `\t` 탭으로 구분된 텍스트 파일
- 각 파일의 첫 번째 줄은 헤더이고, 한 줄이 레코드 1개에 해당

### 데이터 집합 속성

웹사이트: https://s3.amazonaws.com/amazon-reviews-pds/readme.html

이 데이터 집합은 Amazon의 승인에 따라 제공되며, AWS 디지털 교육 서비스 약정(https://aws.amazon.com/training/digital-training-agreement 참조)이 적용됩니다. 본 실습을 완료하는 것 외의 다른 목적으로 이 데이터 집합의 복사, 수정, 판매, 내보내기 또는 사용은 명시적으로 금지됩니다.

## 브레인스토밍 및...

...기계 학습으로 대답할 수 있는 질문 설계 

대부분의 프로젝트에서 첫 번째 단계는 물어보고 싶은 질문, 사용 가능한 데이터가 어떻게 이 질문을 지원하는지, 그리고 질문에 대답하는 데 사용할 도구(이 예제에서는 기계 학습 모형)에 대해 생각하는 것입니다. 이는 탐색 범위를 좁히고 사용할 특성을 명확히 하는 데 도움이 되기 때문에 중요한 단계입니다. 

잠시 시간을 내어 데이터 집합에 대한 생각을 아래 셀에 적어보십시오. 기계 학습으로 예측할 수 있는 것은 무엇인가요? 이것이 비즈니스/고객 관점에서 관련이 있는 이유는 무엇일까요? 왜 이런한 생각을 중요하게 생각하는지 설명하세요.

In [ ]:
# 여기에 의견 작성

데이터로 무엇을 할지 여러 가지 아이디어가 있을 수 있지만, 지금은 특정 사용자에게 비디오를 추천하는 작업에만 전념할 것입니다.

## 추천 및 Factorization Machine

여러 면에서 추천 시스템은 기계 학습이 현재 인기를 얻는 데 촉진제 역할을 했습니다. Amazon의 초창기 성공 사례 중 하나는 "이 상품을 구입한 고객이 ...도 구입했습니다"라는 기능이었습니다. 백만 달러 규모의 Netflix Prize는 연구의 원동력이 되었고, 대중의 인식을 높였으며, 다른 수많은 데이터 과학 대회에 영감을 주었습니다.

추천 시스템은 수많은 데이터 소스와 기계 학습 알고리즘을 활용할 수 있습니다. 대부분은 다양한 비지도, 지도 및 강화 학습 기법을 전체론적 프레임워크로 결합합니다. 하지만 핵심 구성 요소는 유사 항목에 대한 어떤 사용자의 평가 기록은 물론 다른 유사한 사용자의 행동을 기반으로 특정 항목에 대한 해당 사용자의 평점(또는 구매)을 예측하는 모형입니다. 이를 위해 필요한 최소한의 데이터 집합은 사용자의 항목 평점 기록이며 이 데이터는 우리가 보유하고 있습니다.

사용할 기법은 Factorization Machine입니다. Factorization Machine은 분류 문제와 회귀 문제 모두에 사용할 수 있는 범용 지도 학습 알고리즘입니다. 이는 선형 모형의 확장이며, 고차원 희박 데이터 집합 내 특성 간 상호 작용을 경제적으로(간단하게) 캡처하도록 설계되었습니다. 따라서 클릭 예측 및 항목 추천과 같은 특성이 있는 데이터 패턴을 처리하기에 적합합니다.

# 1단계: 문제 공식화 및 데이터 수집

이 시나리오의 비즈니스 문제와 달성하고자 하는 비즈니스 목표를 몇 문장으로 요약하여 아래에 작성하는 것으로 이 프로젝트를 시작하십시오. 팀에서 지향했으면 하는 비즈니스 지표를 포함합니다. 지표를 정의한 후 기계 학습 문제 기술서를 명확하게 작성합니다. 마지막으로, 이러한 문제에 해당하는 기계 학습의 유형에 대해 설명을 한 두 개 추가합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 프로젝트 프레젠테이션에 이러한 세부 정보에 대한 요약을 포함합니다.</span>

### 비즈니스 시나리오를 읽고 다음을 수행합니다.

### 1. 기계 학습이 배포에 적합한 솔루션인지와 그 이유 파악

In [ ]:
# 여기에 답변 작성

### 2. 비즈니스 문제, 성공 지표 및 원하는 기계 학습 결과 공식화

In [ ]:
# 여기에 답변 작성

### 3. 해결하려는 기계 학습 문제의 유형 식별

In [ ]:
# 여기에 답변 작성

### 4. 작업 중인 데이터의 적합성 분석

In [ ]:
# 여기에 답변 작성

### 설정

이제 어디에 에너지를 집중해야 할지 결정했으니, 문제 해결에 착수할 수 있도록 환경을 설정해 보겠습니다.

**참고:** 이 노트북은 `ml.m4.xlarge` 노트북 인스턴스에서 생성 및 테스트되었습니다. 

먼저 다음을 지정하여 시작하겠습니다.
- 훈련 및 모형 데이터에 사용할 Amazon Simple Storage Service(S3) 버킷 및 접두사(?). 노트북 인스턴스, 훈련 및 호스팅과 동일한 리전 내에 있어야 합니다.
- 데이터에 대한 훈련 및 호스팅 액세스 권한을 부여하는 데 사용되는 AWS Identity and Access Management(IAM) 역할 [Amazon 리소스 이름(ARN)](https://docs.aws.amazon.com/general/latest/gr/aws-arns-and-namespaces.html). 이러한 항목을 생성 방법은 설명서를 참조하십시오.

**참고:** 노트북 인스턴스, 훈련 및/또는 호스팅에 둘 이상의 역할이 필요한 경우 `get_execution_role()` 호출을 적절한 전체 IAM 역할 ARN 문자열로 대체하십시오.

***`<LabBucketName>`**을(를) 실습 계정에 제공된 리소스 이름으로 바꿉니다.

In [ ]:
# 정보에 따라 버킷 및 접두사 변경
bucket = '<LabBucketName>'
prefix = 'sagemaker-fm' 

import sagemaker
role = sagemaker.get_execution_role()

이제 이 예제 노트북의 나머지 부분에 필요한 Python 라이브러리를 로드합니다.

In [ ]:
import os, subprocess
import warnings
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.mxnet import MXNet
import boto3
import json
import matplotlib.pyplot as plt
import seaborn as sns

# 마지막 출력뿐만 아니라 셀의 모든 출력을 표시하려면 다음을 추가합니다.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 경고 무시
warnings.filterwarnings("ignore")

# 2단계: 데이터 전처리 및 시각화 
이 데이터 전처리 단계에서는 데이터를 더 잘 이해할 수 있도록 데이터를 탐색하고 시각화할 수 있는 기회를 가져야 합니다. 먼저 필요한 라이브러리를 가져와서 데이터를 Pandas 데이터 프레임으로 읽어 들입니다. 그런 다음 데이터를 탐색합니다. 데이터 집합의 형태를 찾고 열과 작업 중인 열 유형(수치형, 범주형)을 탐색합니다. 특성에 대한 기본 통계를 수행하여 특성의 평균 및 범위를 파악하는 것이 좋습니다. 목표 열을 자세히 살펴보고 분포를 확인합니다.

### 고려해야 할 구체적인 질문
1. 특성에 대해 실행한 기본 통계에서 추론할 수 있는 것은 무엇인가요? 

2. 목표 클래스의 분포에서 추론할 수 있는 것은 무엇인가요?

3. 데이터를 탐색하면서 추론한 것이 또 있나요?

### <span style="color: blue;">프로젝트 프레젠테이션: 이러한 질문과 기타 유사한 질문에 대한 답변을 요약하여 프로젝트 프레젠테이션에 포함합니다.</span>

먼저 Amazon S3 퍼블릭 버킷에서 이 노트북 환경으로 데이터 집합을 가져옵니다.

In [ ]:
# 파일이 이미 원하는 경로에 있는지 아니면 다운로드해야 하는지 확인합니다.

base_path = '/home/ec2-user/SageMaker/project/data/AmazonReviews'
file_path = '/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz'

if not os.path.isfile(base_path + file_path):
    subprocess.run(['mkdir', '-p', base_path])
    subprocess.run(['aws', 's3', 'cp', 's3://amazon-reviews-pds/tsv' + file_path, base_path])
else:
    print('File already downloaded!')

### 데이터 집합 읽어 들이기

어떤 데이터를 다루고 있는지 알 수 있도록 데이터를 Pandas 데이터 프레임으로 읽어 들입니다.

**참고:** 파일을 읽어 들일 때 `error_bad_lines=False`라고 설정합니다. 그렇지 않으면 적은 수의 레코드가 문제를 일으킬 수 있기 때문입니다.

**힌트:** 기본 제공 Python 함수인 'read_csv'를 사용할 수 있습니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)). `delimiter='\t'`를 이용해 Pandas `read_csv`에서 파일 경로를 직접 사용할 수 있습니다.

예: `pd.read_csv('filename.tar.gz', delimiter = '\t', error_bad_lines=False)`

In [ ]:
df = # 여기에 코드 입력

데이터 집합의 처음 몇 행을 인쇄합니다.  

**힌트**: `pandas.head(<number>)` 함수를 사용하여 행을 인쇄합니다.

In [ ]:
# 여기에 코드 입력

이제 모든 열에 포함된 정보는 무엇인가요?

### 데이터 집합의 구조

데이터에 좀 더 익숙해지고 현재 어떤 특성이 있는지 확인하십시오.

- `marketplace`: 2자 국가 코드(이 예제에서는 모두 "US")
- `customer_id`: 단일 작성자가 작성한 리뷰를 집계하는 데 사용할 수 있는 랜덤 식별자
- `review_id`: 리뷰의 고유 ID
- `product_id`: ASIN(아마존 표준 식별 번호. http://www.amazon.com/dp/&lt;ASIN\>는 제품 세부 정보 페이지로 연결됩니다.
- `product_parent`: 해당 ASIN의 상위 상품입니다. 여러 개의 ASIN(동일한 상품의 색상 또는 형식 변형)이 단일 상위 상품으로 겹쳐서 표시될 수 있습니다.
- `product_title`: 상품의 타이틀 설명
- `product_category`: 리뷰 그룹화에 사용할 수 있는 광범위한 상품 카테고리(이 예제에서는 디지털 비디오)
- `star_rating`: 상품 평점(별 1개~5개)
- `helpful_votes`: 리뷰가 받은 “도움 돼요” 투표 수
- `total_votes`: 리뷰가 받은 “도움 돼요” 총 투표 수
- `vine`: 리뷰는 Vine 프로그램의 일부로 작성되었나요?
- `verified_purchase`: 확인된 구매의 리뷰인가요?
- `review_headline`: 리뷰 자체의 제목
- `review_body`: 리뷰 내용
- `review_date`: 리뷰가 작성된 날짜

### 데이터 집합 분석 및 이해

#### 데이터 탐색

**힌트:** [여기](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html)를 참조하여 다음 질문에 답할 수 있습니다. 

**질문:** 데이터 집합에 행과 열이 몇 개 있나요?

데이터 집합의 크기를 확인합니다.  

**힌트**: `<dataframe>.shape` 함수를 사용하여 데이터 프레임의 크기를 확인합니다.

In [ ]:
# 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

**질문:** 어떤 열에 null 값이 포함되어 있으며 이러한 열에는 얼마나 많은 null 값이 포함되어 있나요?

데이터 집합의 요약을 인쇄합니다.  

**힌트**: `<dataframe>.info` 함수에 키워드 인수 `null_counts = True`를 사용합니다.

In [ ]:
# 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

**질문:** 중복되는 행이 있나요? 만약 그렇다면 몇 개가 있나요?  

**힌트**: `dataframe.duplicated()` ([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated))를 사용하여 데이터 프레임을 필터링하고 새로운 데이터 프레임의 길이를 확인합니다.

In [ ]:
duplicates = # 여기에 코드 입력

# 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

### 데이터 전처리

이제 사용할 특성을 결정하고 모형에 맞게 해당 특성을 준비할 방법을 결정할 차례입니다. 이 예제에서는 특성을 `customer_id`, `product_id`, `product_title` 및 `star_rating`으로 제한합니다. 추천 시스템에 추가 특성을 포함하면 유용할 수 있지만, 상당한 처리 작업(특히 텍스트 데이터)이 필요하며 이는 이 노트북의 범위를 벗어나는 것입니다.

이 데이터 집합을 줄이고 언급된 열만 사용합니다.  

**힌트**: 열을 목록으로 전달하여 여러 열을 데이터 프레임으로 선택합니다. 예: `df[['column_name 1', 'column_name 2']]`

In [ ]:
df_reduced = # 여기에 코드 입력

데이터 집합을 줄인 후 중복이 있는지 다시 확인합니다. 

In [ ]:
duplicates = # 여기에 코드 입력

# 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

**질문:** 지금 데이터 집합에 중복이 있는 이유는 무엇일까요? 데이터 집합을 줄인 후 무엇이 바뀌었나요? 중복이 있는 처음 20줄을 검토합니다. 

**힌트**: `pandas.head(<number>)` 함수를 사용하여 행을 인쇄합니다.

In [ ]:
# 여기에 코드 입력

**힌트:** 중복 데이터 프레임의 처음 두 요소를 살펴보고 원본 데이터 프레임 df를 쿼리하여 데이터가 어떤 모습인지 확인합니다. `query` 함수를 사용할 수 있습니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)).

예:

```
df_eg = pd.DataFrame({
            'A': [1,2,3,4],
            'B': [
        })
df_eg.query('A > 1 &amp; B > 0')
```

In [ ]:
# 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

계속하기 전에 중복 행을 제거합니다.

**힌트**: `~` 연산자를 사용하여 중복되지 않은 행을 모두 선택합니다. 예:
    
```
df_eg = pd.DataFrame({
            'A': [1,2,3,4],
            'B': [2,0,5,2]
        })
df_eg[~(df_eg['B'] > 0)]
```

In [ ]:
df_reduced = # 여기에 코드 입력

### 데이터 집합의 일부 행 시각화
위에서 시각화를 수행하지 않은 경우 아래 공간을 사용하여 일부 데이터를 추가로 시각화할 수 있습니다. 구체적으로 `star_rating`, `customer_id` 및 `product_id`와 같은 특성의 분포를 살펴봅니다.

**고려해야 할 구체적인 질문**

1. 특성의 분포를 살펴보니 이러한 특성이 모형에 어느 정도 도움이 될까요? 이러한 분포에서 데이터를 더 잘 이해하는 데 도움이 되도록 추론할 수 있는 것이 있나요? 

2. 모든 데이터를 사용해야 할까요? 어떤 특성을 사용해야 할까요?

3. 어느 달에 사용자 평가 수가 가장 많은가요?

아래 셀을 사용하여 데이터를 시각화하고 이러한 질문과 관심을 가질만한 다른 질문에 답하십시오. 필요에 따라 셀을 삽입하고 삭제합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 이러한 질문과 유사한 질문에 대한 답변을 요약하여 프로젝트 프레젠테이션에 포함합니다.</span>

`sns.barplot`([설명서](https://seaborn.pydata.org/generated/seaborn.barplot.html))을 사용하여 `star_rating` 밀도 및 분포를 표시합니다.

In [ ]:
# 특정 평점의 리뷰 수를 계산하려면 여기에 코드 입력

sns.barplot(
    x='index', 
    y =<CODE>, # 여기에 코드 입력
    data=_, # Python의 밑줄 기호는 마지막 연산의 출력을 저장하는 데 사용됩니다.
    palette='GnBu_d'
)

**질문:** 어느 달에 사용자 평가 수가 가장 많은가요?  

**힌트**:  
1. `pd.to_datetime`을 사용하여 `review_date` 열을 날짜/시간 열로 변환합니다.  
2. `review_date` 열의 월을 사용합니다. `<column_name>.dt.month`를 사용하여 날짜/시간 열에 액세스할 수 있습니다.  
3. `idxmax`를 `groupby` 함수에 사용합니다.  

In [ ]:
# 리뷰 날짜를 날짜/시간 유형으로 변환합니다. 여기서는 원본 데이터 프레임인 'df'를 사용합니다.
df['review_date'] = # 여기에 코드 입력

# 월별로 리뷰 수 계산
df.groupby(<CODE>).star_rating.count().reset_index()

# 다시 막대 플롯을 사용하여 ratings(y) 대 review_date(x)를 표시합니다.
sns.barplot(x=<CODE>, y=<CODE>, data=_, palette='GnBu_d') # 여기에 코드 입력

In [ ]:
# 월에 Pandas groupby 함수를 사용하여 별 평점 수 구하기
max_month = df.groupby(<CODE>).star_rating.count().idxmax() # 여기에 코드 입력
print(f'The month with the most reviews is: {max_month}')

In [ ]:
# 여기에 답변 입력

**보너스 질문(선택 사항):** 리뷰가 가장 많은 연도와 가장 적은 연도는 언제인가요?

In [ ]:
# 연도에 Pandas groupby 함수를 사용하여 별 평점 수 구하기
df.groupby(<CODE>).star_rating.count().reset_index() # 여기에 코드 입력

fig = plt.gcf()
fig.set_size_inches(10, 5)

# 막대 플롯을 사용하여 star_rating(y) 대 review_date(x)를 표시합니다.
sns.barplot(x=<CODE>, y=<CODE>, data=_, palette='GnBu_d') # 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

### 데이터 정제

**질문**: 고객당 리뷰 수와 비디오당 리뷰 수는 얼마나 다른가요? 사분위수를 사용하여 확인합니다.

**힌트**: 고객과 상품 데이터 프레임에는 `<dataframe>['columns_name'].value_counts()`를 사용하고 `<dataframe>.quantile(<list>)`을 사용하여 관계를 찾습니다.

In [ ]:
customers = # 여기에 코드 입력
products = # 여기에 코드 입력

quantiles = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25, 0.5, 
             0.75, 0.9, 0.95, 0.96, 0.97, 0.98, 0.99, 0.995, 
             0.999, 1]
print('customers\n', <CODE>) # 여기에 코드 입력
print('products\n', <CODE>) # 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

이 롱 테일을 필터링합니다. 18개 이상의 비디오에 평점을 매긴 고객과 95개가 넘는 리뷰가 있는 제품을 선택합니다. 

In [ ]:
customers1 = # 여기에 코드 입력
products1 = # 여기에 코드 입력

# Pandas merge 함수를 사용하여 customer1 및 products1을 원본 df_reduced 데이터 집합과 병합
reduced_df = (
            df_reduced.merge(pd.DataFrame({'customer_id': customers1.index}))
                      .merge(pd.DataFrame({'product_id': products1.index}))
            )# 여기에 코드 입력

**질문:** '`customers1`, `products1` 및 새로운 데이터 프레임 reduced_df는 어떤 형태인가요?  

**참고**: 여기에는 f-strings를 사용합니다.
```
x= 3
print(f'X = {x}')
```

In [ ]:
print(f'Number of users is {<CODE>} and number of items is {<CODE>}.')# 여기에 코드 입력
print(f'Length of reduced df is {<CODE>}.')# 여기에 코드 입력

데이터 프레임의 처음 5개 열을 인쇄합니다.

In [ ]:
# 여기에 코드 입력

**질문:** `reduced_df`는 동일한 평점 비율을 유지하나요?

In [ ]:
reduced_df[<CODE>].value_counts().reset_index()# 여기에 코드 입력
sns.barplot(x='index', y='star_rating', data=_, palette='GnBu_d')

In [ ]:
# 여기에 답변 입력

이제 고객 및 제품별 개수의 고객 및 제품 배포를 다시 만듭니다.

**힌트**: `customer_id` 및 `product_id` 열에 'v`value_counts()` 함수를 사용합니다.

In [ ]:
customers = # 여기에 코드 입력
products = # 여기에 코드 입력

fig, axs = plt.subplots(1, 2, figsize=(20, 5))
fig.suptitle('Distribution of counts per customer and product')
sns.distplot(customers, kde=False, ax=axs[0], color='teal')
sns.distplot(products, kde=False, ax=axs[1])

그런 다음 각 사용자 및 항목에 번호를 붙여 고유한 순차 인덱스를 부여합니다. 이렇게 하면 순차 인덱스가 평점 행렬의 행과 열을 나타내는 희박 형식으로 정보를 저장할 수 있습니다.

`customer_index` 및 `product_index를 생성하려면 `customer_id`를 인덱스 값으로 사용하고 사용자 및 항목 번호에 순차 카운터/값을 사용하여 새로운 데이터 프레임을 생성합니다. 두 인덱스를 모두 생성했으면 Pandas `merge` 함수를 사용하여 `customer_index`와 `product_index`를 병합합니다.

**힌트**: `shape` 함수를 사용하여 총 고객 및 제품 수를 생성합니다. `np.arange`를 사용하여 0부터 고객 및 제품 수까지의 숫자 목록을 생성합니다.

In [ ]:
customer_index = pd.DataFrame({'customer_id': customers.index, 
                               'user': np.arange(<CODE>)}) # 여기에 코드 입력
product_index = pd.DataFrame({'product_id': products.index, 
                              'item': np.arange(<CODE>)}) # 여기에 코드 입력

reduced_df = reduced_df.merge(<CODE>).merge(<CODE>)# 여기에 코드 입력
reduced_df.head()

샘플 답변:
<div class="output_subarea"><div>

<table class="dataframe" border="1">
  <thead>
    <tr style="text-align: right">
      <th></th>
      <th>customer_id</th>
      <th>product_id</th>
      <th>star_rating</th>
      <th>product_title</th>
      <th>user</th>
      <th>item</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>11763902</td>
      <td>B00PSLQYWE</td>
      <td>4</td>
      <td>Downton Abbey Season 5</td>
      <td>3065</td>
      <td>103</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1411480</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>Downton Abbey Season 5</td>
      <td>130</td>
      <td>103</td>
    </tr>
    <tr>
      <th>2</th>
      <td>35303629</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>Downton Abbey Season 5</td>
      <td>4683</td>
      <td>103</td>
    </tr>
    <tr>
      <th>3</th>
      <td>21285980</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>Downton Abbey Season 5</td>
      <td>449</td>
      <td>103</td>
    </tr>
    <tr>
      <th>4</th>
      <td>29260449</td>
      <td>B00PSLQYWE</td>
      <td>5</td>
      <td>Downton Abbey Season 5</td>
      <td>131</td>
      <td>103</td>
    </tr>
  </tbody>
</table>
</div></div>

## <span style="color:red"> 실습 2 종료 </span>

프로젝트 파일을 로컬 컴퓨터에 저장합니다. 방법은 다음과 같습니다.

1. 페이지 상단에 있는 **파일** 메뉴를 클릭합니다. 

1. **다운로드**를 선택하고 **Notebook(.ipynb)**을 클릭합니다.  

그러면 현재 노트북이 컴퓨터의 기본 다운로드 폴더로 다운로드됩니다.

# 3단계: 모형 훈련 및 평가

데이터 프레임에서 기계 학습 알고리즘이 사용할 수 있는 형식으로 데이터 집합을 변환할 때 포함해야 하는 몇 가지 예비 단계가 있습니다. Amazon SageMaker의 경우 다음 단계를 수행해야 합니다.

1. 데이터를 `train_data` 및 `test_data`로 분할합니다.    
2. 데이터 집합을 Amazon SageMaker 훈련 작업에서 사용할 수 있는 적절한 파일 형식으로 변환합니다. CSV 파일 또는 레코드 protobuf로 변환하면 됩니다. 자세한 내용은 [훈련을 위한 공통 데이터 형식](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html) 페이지를 참조하십시오. 이 문제의 경우 데이터가 희박 데이터이므로 `scipy.sparse.lilmatrix` 함수를 사용한 다음 `sagemaker.amazon.common.write_spmatrix_to_sparse_tensor`를 사용하여 함수를 `RecordIO protobuf` 형식으로 변환하면 됩니다.    
3. 데이터를 Amazon S3 버킷에 업로드합니다. 이전에 버킷을 생성한 적이 없는 경우 [버킷 생성](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html) 페이지를 참조하십시오.    

다음 셀을 사용하여 이러한 단계를 완료합니다. 필요에 따라 셀을 삽입하고 삭제합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 이 단계에서 내린 주요 의사 결정을 프로젝트 프레젠테이션에 기록해 둡니다.</span>

### 데이터 준비

이제 모형의 입력으로 데이터 집합을 준비하기 시작할 수 있습니다. 모형마다 입력 요구 사항이 다릅니다. Amazon SageMaker에 구현된 일부 알고리즘에서는 데이터가 recordIO-wrapped protobuf 형식이어야 합니다. 다음 셀에서 이를 처리합니다.

먼저 데이터 집합을 훈련 집합과 검정 집합으로 분할합니다. 이렇게 하면 고객이 평점을 주었지만 훈련에는 포함되지 않은 비디오를 통해 모형의 정확도를 추정할 수 있습니다.

먼저 `test_df` 데이터 프레임을 생성합니다. 데이터 프레임을 `customer_id`로 그룹화하고 `pd.groupby(' ').last()`와 유사한 `last` 함수를 사용하여 데이터 프레임을 생성합니다.

In [ ]:
test_df = reduced_df.groupby(<CODE>).last().reset_index() # 여기에 코드 입력

훈련 데이터를 생성하려면 `reduced_df` 데이터 프레임에서 `test_df`에 있는 값을 제거합니다.

**힌트**: `reduced_df` 데이터 프레임을 `customer_id` 및 `product_id` 열이 있는 `test_df` 데이터 집합과 외부 조인으로 병합합니다.

In [ ]:
# 여기에 코드 입력
train_df = reduced_df.merge(<CODE>, 
                            on=['customer_id', 'product_id'], 
                            how='outer', 
                            indicator=True, 
                            indicator=True)
train_df = train_df[(train_df['_merge'] == 'left_only')].reset_index()

In [ ]:
test_df.head()

이제 데이터의 몇 가지 기본 특징을 살펴보겠습니다. 이는 나중에 특성을 모형 훈련에 적합한 형식으로 변환하는 데 도움이 될 것입니다.

검정 데이터 집합과 훈련 데이터 집합의 길이에 대해 각각 `nb_rating_test` 및 `nb_ratings_train`이라는 변수 두 개를 생성합니다.

In [ ]:
nb_ratings_test = # 여기에 코드 입력
nb_ratings_train = # 여기에 코드 입력
print(f" Training Count: {nb_ratings_train}")
print(f" Test Count: {nb_ratings_test}")

### 데이터 변환

이제 Pandas 데이터 프레임을 희박 행렬로 변환할 수 있습니다. 이 프로세스는 훈련 및 검정 모두에서 동일합니다. Amazon SageMaker에서 Factorization Machine을 구현하려면 recordIO-wrapped protobuf가 필요한 데 현재 디스크에 있는 데이터는 Pandas 데이터 프레임입니다. 따라서 데이터를 희박 행렬로 변환하여 각 사용자와 각 영화 간의 관계를 표현해야 합니다.

In [ ]:
from scipy.sparse import lil_matrix

def loadDataset(df, lines, columns, regressor=True):
    """
    Pandas 데이터 프레임을 희박 행렬로 변환
    
    Args:
        df: DataFrame
        lines: 최종 희박 행렬의 행 수
        columns: 최종 희박 행렬의 열 수
        regressor: 회귀 또는 분류 사용 여부를 확인하는
                  부울 값
    Returns:
        X: 특성 벡터
        Y: 레이블 벡터
    """
    # 특성은 희박 행렬에 One-Hot 인코딩됩니다.
    
    # scipy.sparse.lil_matrix를 사용하여 float32 유형의 특성 벡터 X 생성
    # 행렬의 크기는 데이터 프레임의 길이와 
    # 줄 수+열 수 변수 
    X = lil_matrix((<CODE>, lines + columns)).astype('float32') # 여기에 코드 입력
    
    # 레이블은 벡터에 저장됩니다. 빈 레이블 벡터 Y를 인스턴스화합니다.
    Y = # 여기에 코드 입력
    
    line = 0
    
    # 데이터 프레임의 각 행에 대해 항목 및 제품 번호에 1을 사용합니다.
    for index, row in df.iterrows():
        X[line,row['user']] = 1
        X[line, lines + (row['item'])] = 1
        line += 1

        if regressor:
            # 회귀를 사용하는 경우 행 변수에서 star_rating 추가
            Y.append(<CODE>) # 여기에 코드 입력
        else:
            # 행 변수에서 star_rating이 5이면 1을 사용하고 그 외에는 0을 사용
            if int(row['star_rating']) >= 5:
                Y.append(<CODE>) # 여기에 코드 입력
            else:
                Y.append(<CODE>) # 여기에 코드 입력
            
    # 목록을 float32 유형의 NumPy 배열로 변환     
    Y = np.array(<CODE>).astype('float32') # 여기에 코드 입력
    
    return X, Y

`loadDataset` 함수를 사용하여 훈련 집합과 검정 집합을 생성합니다.

In [ ]:
print(customers.shape[0], 
      products.shape[0],
      customers.shape[0] + products.shape[0])

# train_df, customers.shape[0] and products.shape[0]와 함께 loadDataset 함수 사용
X_train, Y_train = loadDataset(<CODE>) # 여기에 코드 입력

# test_df, customers.shape[0] and products.shape[0]와 함께 loadDataset 함수 사용
X_test, Y_test = loadDataset(<CODE>) # 여기에 코드 입력

이제 데이터가 희박 형식이므로 protobuf 형식으로 저장하고 Amazon S3에 업로드합니다. 이 단계는 어렵게 보일 수 있지만, 대부분의 변환 작업은 아래와 같이 SageMaker로 가져온 Amazon SageMaker Python SDK에서 처리합니다.

In [ ]:
import io 
import sagemaker.amazon.common as smac

def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)


fm_train_data_path = writeDatasetToProtobuf(X_train, bucket, prefix, 'train', "sparse", Y_train)    
fm_test_data_path = writeDatasetToProtobuf(X_test, bucket, prefix, 'test', "sparse", Y_test)  
  
print("Training data S3 path: ", fm_train_data_path)
print("Test data S3 path: ", fm_test_data_path)

이렇게 해서 데이터 준비가 완료되었습니다. 수고하셨습니다! 보시다시피 모형화를 위해 데이터를 정제하고 준비하는 데는 많은 시간과 노력이 필요합니다. 이는 모든 데이터 과학 프로젝트에 해당하며, 이 단계는 결과에 큰 영향을 미칩니다. 앞으로 모든 기계 학습 프로젝트에서 데이터를 이해하고 훈련을 위해 데이터를 준비하는 데 충분한 시간을 할애하시기 바랍니다!

## 모형 훈련

이제 모형을 훈련할 차례입니다. 이를 위해 Amazon SageMaker 훈련 작업을 사용합니다. Amazon SageMaker 훈련 작업을 사용하면 사용자가 훈련을 위해 모든 코드를 작성할 필요가 없으므로 모형을 쉽게 생성할 수 있습니다. 이는 이미 컨테이너 형식으로 처리되어 있습니다.

노트북에서는 예측 변수를 인스턴스화하고, 일부 하이퍼파라미터를 전달한 다음, 데이터를 올바른 형식으로 전달하는 것이 훈련 작업을 생성하는 일반적인 워크플로입니다. 다음 셀에서 이러한 작업이 이루어집니다.

FM 예측 변수에 대한 자세한 내용은 [FactorizationMachines](https://sagemaker.readthedocs.io/en/stable/factorization_machines.html) 페이지를 참조하십시오.

하이퍼파라미터에 대한 자세한 내용은 [Factorization Machine 하이퍼파라미터](https://docs.aws.amazon.com/sagemaker/latest/dg/fact-machines-hyperparameters.html) 페이지를 참조하십시오.

**힌트**: 예:

```
sess = sagemaker.Session()

pca = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role,
                                    train_instance_count=1,
                                    train_instance_type='ml.c4.xlarge',
                                    output_path=output_location,
                                    sagemaker_session=sess)
                                    
pca.set_hyperparameters(featuer_dim=50000,
                        num_components=10,
                        subtract_mean=True,
                        algorithm_mode='randomized',
                        mini_batch_size=200)
                        
pca.fit({'train': s3_train_data})
```

In [ ]:
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

output_prefix = 's3://' + bucket + '/sagemaker-fm/model'
instance_type= # 여기에 코드 입력
batch_size = # 여기에 코드 입력

fm = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, "factorization-machines"),
    role, 
    train_instance_count=<CODE>, # 여기에 코드 입력
    train_instance_type=instance_type,
    output_path=output_prefix,
    sagemaker_session=sagemaker.Session()
)

# 하이퍼파라미터 사용. feature_dim의 경우 X_train 열 길이를 사용합니다. 
fm.set_hyperparameters(
                        feature_dim=<CODE>, # 여기에 코드 입력
                        predictor_type='regressor',
                        mini_batch_size=batch_size,
                        num_factors=64,
                        epochs=25,
                        clip_gradient=5.0,
                        rescale_grad=1.0/batch_size
)

fm.fit({'train': ,# 여기에 코드 입력, 
        'test': # 여기에 코드 입력
       })

**질문:** '`batch_size`와 `epochs`를 변경하면 최종 지표에 어떤 영향을 미칠까요?  

In [ ]:
# 여기에 답변 입력

**질문:** 모형의 결과를 확인합니다. 사용된 지표의 의미는 무엇인가요? 훈련 집합과 검정 집합 간에 차이가 있나요? 차이가 있다면 그 의미는 무엇일까요?  

In [ ]:
# 여기에 답변 입력

### 평가

축하합니다! Amazon SageMaker 훈련 작업을 성공적으로 시작했습니다. 다음 단계는 무엇일까요? 모형이 실제로 일관된 값을 예측하고 있는지 확인할 수 있는 방법이 필요합니다. 어떻게 하면 될까요?

모형의 성능이 어떤지 대략적으로 파악하기 위해 단순 기준선을 계산하는 것부터 시작합니다. 가장 간단한 추정은 모든 사용자 항목 평점이 모든 평점에 대한 평균 평점이라고 가정하는 것입니다. 이는 기본적으로 모든 리뷰의 평균값을 출력하는 방법만 학습한 모형을 가지고 있음을 의미합니다.

**참고:** 각 개별 비디오의 평균을 사용하면 더 나은 결과를 얻을 수 있습니다. 하지만 이 경우에는 동일한 결론이 유지되기 때문에 사실 중요하지 않습니다.

`star_ring`의 평균을 계산하여 `naive_guess` 값을 얻습니다. 그런 다음 검정 `star_rating`에서 `naive_guess`를 제곱하고 평균을 구하여 단순 MSE를 계산합니다.

$average(test(star\_rating) - naive\_guess)^2)$

In [ ]:
naive_guess = np.mean(<CODE>) # 여기에 코드 입력
print(f'Naive MSE:', np.mean((<CODE>)**2)) # 여기에 코드 입력)

이제 검정 데이터 집합에 대한 예측을 계산합니다. 이를 위해서는 방금 훈련한 모형을 _deploy_해야 합니다.

**참고:** 이는 위의 CloudWatch 출력과 긴밀하게 일치하지만 `eval_net` 함수에서 부분 미니 배치를 건너뛰기 때문에 약간 다를 수 있습니다.

`<estimator_name>.deploy`에 `initial_instance_count=1, instance_type=ml.c5.xlarge`를 사용합니다.

In [ ]:
fm_predictor = # 여기에 코드 입력

이제 엔드포인트가 'InService' 상태이므로 검정 집합에서 모형의 성능이 어떤지 평가합니다. 검정 집합에서의 성능을 훈련 집합에서의 성능과 비교합니다. 

### 고려해야 할 주요 질문:
1. 훈련 집합과 비교하여 검정 집합에서의 모형 성능이 어떤가요? 이 비교에서 무엇을 추론할 수 있나요? 

2. 정확도, 정밀도, 재현율과 같은 지표의 결과 간에 분명한 차이가 있나요? 만약 그렇다면, 이러한 차이가 나타나는 이유는 무엇일까요? 

3. 비즈니스 상황과 목표를 고려래 볼 때 여기서 고려해야 할 가장 중요한 지표는 무엇일까요? 그 이유는 무엇일까요?

4. 가장 중요하다고 생각하는 지표의 결과가 비즈니스 관점에서 필요한 결과로도 충분한가요? 그렇지 않다면, 다음 반복(다음에 나오는 특성 엔지니어링 섹션)에서 무엇을 변경할 수 있을까요? 

아래 셀을 사용하여 이러한 질문과 기타 질문에 답하십시오. 필요에 따라 셀을 삽입하고 삭제합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 이러한 질문과 이 섹션에서 대답할 수 있는 유사한 질문에 대한 답변을 프로젝트 프레젠테이션에 기록합니다. 주요 세부 정보와 결정한 내용을 프로젝트 프레젠테이션에 기록합니다.</span>

배포 프로세스에는 훈련하고 Amazon S3에 저장한 모형으로 지정된 크기의 인스턴스(이 경우 `ml.c4.xlarge`)를 생성하는 작업이 포함됩니다. 예측을 얻으려면 직렬화된 JSON 형식으로 데이터를 전달해야 합니다. 추론에서 가져오는 출력도 직렬화된 JSON 형식이므로 역직렬화해야 예측된 값을 얻을 수 있습니다.

In [ ]:
# 예측 변수에 대한 직렬 변환 함수 생성
import json
from sagemaker.predictor import json_deserializer

def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

훈련 집합의 성능이 어땠는지 확인합니다. 엔드포인트를 사용하여 모형에서 예측을 가져옵니다.

먼저 단일 예측이 어떻게 되는지 살펴봅니다.

Amazon SageMaker 모형 컨테이너는 60초 이내에 요청에 응답해야 합니다. 호출에 응답하기 전에 모형 자체에서 처리하는 데 최대 60초까지 걸릴 수 있기 때문입니다. 이렇게 하려면 한 번에 5개 행에 대해 `predict` 함수를 호출한 다음 해당 행을 목록에 추가합니다. 

In [ ]:
# X_train 데이터를 배포된 예측 변수에 전달 
ytrain_p = []
for i in range(0, 1000, 5):
    preds = fm_predictor.predict(<CODE>)<CODE> # 여기에 코드 입력
    p = [ytrain_p.append(x['score']) for x in preds]

**질문:** 추론이 나왔으니 이제 온전성 검사를 수행합니다. 추론에서 예측되는 최소값 및 최대값은 얼마인가요? 훈련 데이터의 최소값 및 최대값과 일치하나요?

In [ ]:
print('The minimum rating predicted is: ', <CODE>, # 여기에 코드 입력
      'and the maximum is: ', <CODE> # 여기에 코드 입력
     )

이제 검정 데이터 집합을 확인합니다.

In [ ]:
Y_pred = []
# 여기에 코드 입력

**질문:** 예측의 최소값과 최대값은 얼마나 비슷한가요? 전체 분포(히스토그램)를 확인하는 경우의 보너스 포인트입니다.

In [ ]:
max(Y_pred), min(Y_pred)

In [ ]:
sns.distplot(Y_pred, kde=False, bins=4)

마지막으로 검정 집합의 평균 제곱근 오차를 계산하고 기준선에서 얼마나 개선되었는지 확인합니다.

In [ ]:
print('MSE:', <CODE> )# 여기에 코드 입력

추천 시스템에서는 주관적 정확도 또한 중요합니다. 랜덤 사용자를 위한 추천을 생성하여 직관력이 있는지 확인합니다.

사용자 번호 200을 사용해 무엇을 시청했고 무엇에 높은 평점을 주었는지 확인합니다.

In [ ]:
reduced_df[<CODE>].sort_values(
    ['star_rating', 'item'], ascending=[False, True]) # 여기에 코드 입력

보시다시피 이 사용자는 코미디, 로맨스 및 가벼운 영화 보는 것을 좋아하며 드라마와 판타지 영화를 싫어합니다. 모형이 이 사용자의 영화 평점을 어떻게 예측했는지 살펴보겠습니다.

In [ ]:
def prepare_predictions(user_id, number_movies, columns):
    # 훈련 데이터에서와 유사한 희박 매트릭스 생성
    X = il_matrix((<CODE>)).astype('float32')# 여기에 코드 입력
    movie_index_start = columns - number_movies

    # 행렬을 채웁니다. 각 행은 동일한 사용자와 가능한 영화로 채워집니다.
    for row in range(number_movies):
        X[row, user_id - 1] = <CODE> # 여기에 코드 입력
        X[row, movie_index_start + row] = <CODE> # 여기에 코드 입력

    return X

user_200 = prepare_predictions(200, 
                               <CODE> # 여기에 코드 입력, 
                               <CODE> # 여기에 코드 입력
                              )

이제 모형은 사용자 200이 모든 영화에 매기는 평점을 예측하는 평점 목록을 생성합니다.

In [ ]:
pred_200 = []
for i in range(0, <CODE>):
    preds = fm_predictor.predict(<CODE>)['predictions']
    p = [pred_200.append(x['score']) for x in preds]

이제 카탈로그에 있는 모든 일반 비디오에 대해 사용자 200의 평점을 반복해서 예측하여 어떤 비디오를 추천 또는 비추천하는지 확인합니다. 

`reduced_df` 데이터 프레임을 사용하여 항목별로 그룹화하여 새로운 데이터 프레임 `titles`를 생성합니다. `product_title` 열을 사용하고 또 하나의 ‘score’ 열을 생성한 후 `pred_200`의 값을 추가합니다.

In [ ]:
titles = reduced_df.groupby(<CODE>)[<CODE>].first().reset_index()
titles['score'] = # 여기에 코드 입력

**질문:* 가장 높은 점수를 받은 비디오는 무엇인가요?  

**힌트**: `sort_values` 함수를 사용하여 `score` 열과 `item` 열을 정렬하고 `asecnding=[False,True]` 파라미터를 사용합니다.

In [ ]:
# 여기에 코드 입력

In [ ]:
# 여기에 답변 입력

**질문:** 사용자가 높은 점수를 준 프로그램과 가장 낮은 점수를 준 프로그램에서 어떤 결론을 내릴 수 있나요? 

In [ ]:
# 여기에 답변 입력

해당 추천이 다른 사용자와 상관 관계가 있는지 확인합니다. 사용자 201을 사용해 확인해 보십시오. 사용자 200에 대해 수행한 것과 동일한 작업을 수행합니다.

In [ ]:
user_201 = prepare_predictions(<CODE>, products.shape[0], customers.shape[0] + products.shape[0])

pred_201 = []
for i in range(0, user_201.shape[0], 5):
    preds = fm_predictor.predict(user_201[i:i+5].toarray())['predictions']
    p = [pred_201.append(x['score']) for x in preds]

In [ ]:
plt.scatter(pred_200, pred_201)
plt.show()

**질문:** 두 사용자 사이의 산점도에서는 어떤 결론을 내릴 수 있나요?  

In [ ]:
# 여기에 답변 입력:

추론을 위해 만든 엔드포인트는 더 이상 사용하지 않으므로 삭제합니다.

In [ ]:
sagemaker.Session().delete_endpoint(fm_predictor.endpoint)

## <span style="color:red"> 실습 3 종료 </span>

프로젝트 파일을 로컬 컴퓨터에 저장합니다. 방법은 다음과 같습니다.

1. 페이지 상단에 있는 **파일** 메뉴를 클릭합니다. 

1. **다운로드**를 선택하고 **Notebook(.ipynb)**을 클릭합니다.  

그러면 현재 노트북이 컴퓨터의 기본 다운로드 폴더로 다운로드됩니다.

# 반복 II

# 4단계: 특성 엔지니어링

이제 모형을 훈련하고 평가하는 과정을 한 번 반복했습니다. 처음 모형을 사용해 도달한 결과로는 비즈니스 문제를 해결하는 데 충분하지 않을 수 있다는 점을 감안할 때 모형 성능을 개선하기 위해 데이터에 대해 변경할 수 있는 사항은 무엇인가요?

### 고려해야 할 주요 질문:
1. 기계 학습 문제를 변경하면 데이터 집합에 어떤 도움이 되나요? 회귀 기법을 사용하여 문제를 해결하려고 했습니다. 분류 기법이 도움이 될까요?
2. 기계 학습 문제를 기계 학습 분류 문제로 바꾸려면 어떻게 해야 하나요? 분류 기법에 맞게 새로운 문제 기술서를 작성합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 이 섹션에서 사용하는 주요 결정 사항과 기법은 물론 모형을 다시 평가한 후 얻은 새로운 성능 지표를 프로젝트 프레젠테이션에 기록합니다.</span>

이제 훈련 데이터 집합이 받은 평점에 따라 이진 출력을 갖도록 변경합니다. 평점이 별 5개일 때 사용자에게 추천하는 것을 고려하고 Amazon S3에 protobuf 형식으로 다시 저장합니다. 다음을 수행합니다.  

1. `loadDataset` 함수를 `regression=False` 옵션과 함께 사용하여 훈련 데이터 집합을 생성합니다.   
2. 데이터 집합을 protobuf 형식으로 작성합니다.  
3. `predictor_type='binary_classifier'`를 사용하여 모형을 다시 훈련합니다.   
4. 엔드포인트에 모형을 배포하고 앞에서 검정 집합으로 했던 것처럼 모형을 평가합니다.   
5. 혼동 행렬을 사용하여 검정 집합에서의 성능을 검사합니다.   

In [ ]:
X_train_class, Y_train_class = # 여기에 코드 입력
X_test_class, Y_test_class = # 여기에 코드 입력

In [ ]:
# 데이터 집합을 protobuf 형식으로 작성
fm_train_data_path = writeDatasetToProtobuf(<CODE>) # 여기에 코드 입력    
fm_test_data_path = writeDatasetToProtobuf(<CODE>) # 여기에 코드 입력    
  
print("Training data S3 path: ", fm_train_data_path)
print("Test data S3 path: ", fm_test_data_path)

### 샘플 코드

```
fm_train_data_path = writeDatasetToProtobuf(X_train_class, bucket, prefix, 'train_class', "sparse", Y_train_class)    
fm_test_data_path = writeDatasetToProtobuf(X_test_class, bucket, prefix, 'test_class', "sparse", Y_test_class) 
```

마지막으로 회귀에서 이진 분류로 변경하여 모형을 다시 훈련합니다. 이전에 모형을 훈련할 때와 동일한 코드 및 설정을 사용하되, `predictor_type='binary_classifier`를 변경합니다.

In [ ]:
# 모형 재훈련
# 여기에 코드 입력

### 샘플 코드 
```
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

#output_prefix= 's3://<LabBucketName>/sagemaker-fm/model'

output_prefix = 's3://' + bucket + '/sagemaker-fm/model'
instance_type='ml.m4.xlarge'
batch_size = 512

fm = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, "factorization-machines"),
    role, 
    train_instance_count=1, 
    train_instance_type=instance_type,
    output_path=output_prefix,
    sagemaker_session=sagemaker.Session()
)

fm.set_hyperparameters(feature_dim=X_train.shape[1],
                     # predictor_type='regressor',
                       predictor_type='binary_classifier',
                       mini_batch_size=batch_size,
                       num_factors=128,
                       epochs=25,
                       clip_gradient=5.0,
                       rescale_grad=1.0/batch_size
                       )

fm.fit({'train': fm_train_data_path, 'test': fm_test_data_path})
```

이 새로운 모형의 성능을 평가합니다. 모형을 배포하고 직렬 변환기를 결정한 다음 검정 데이터를 전달합니다.

In [ ]:
fm_predictor = fm.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

In [ ]:
# 검정 데이터를 classifier에 전달하고 모든 예측을 가져옵니다.
Y_pred = []
for i in range(0, X_test_class.shape[0], 5):
    preds = fm_predictor.predict(X_test_class[i:i+5].toarray())['predictions']
    p = [Y_pred.append(x['score']) for x in preds]

#### 결과 검사

classifier가 얼마나 잘 작동하는지 검사하려면 혼동 행렬을 계산하여 표시합니다. **Scikit-Learn**의 구현을 사용합니다.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
true = Y_test_class.astype(int)
predicted = [1 if value > 0.5 else 0 for value in Y_pred]
conf_matrix = confusion_matrix(true, predicted)
print(conf_matrix)
sns.heatmap(conf_matrix)

**질문:** 모형의 정확도는 어떻게 되나요?  

**힌트**:
$$ Accuracy = \frac{TP + TN}{TP + FP + FN + TN} $$

In [ ]:
# 정확도
# 여기에 코드 입력

**질문:** 여러분의 모형을 모든 것을 1로 예측하는 단순 기준 모형과 어떻게 비교하면 어떤가요?

In [ ]:
(reduced_df.star_rating > 4).value_counts() / reduced_df.shape[0] * 100

In [ ]:
# 여기에 답변 입력

In [ ]:
# 추론 엔드포인트 삭제
sagemaker.Session().delete_endpoint(fm_predictor.endpoint)

## KNN(k-최근접 이웃)과 결합

classifier 모형이 regressor 모형보다 성능이 더 우수하다는 것을 확인했습니다. 평점(regressor)을 예측하거나 사용자가 영화를 좋아하는지 여부(이진 분류)를 예측하는 대신, 이제 KNN(k-최근접 이웃) 모형에 맞게 다시 패키징하여 고객이 좋아하는 것에 *k 최근접* 항목을 예측한 다음 이를 추천할 수 있는지 확인합니다.

먼저 Amazon S3에서 모형을 다운로드합니다. 그런 다음 KNN(k-최근접 이웃) 모형에 맞게 다시 패키징합니다.

**참고:** 다음 셀을 실행할 수 있도록 사용 중인 커널이 `conda_mxnet_p36`인지 확인하십시오.

### 모형 데이터 다운로드

In [ ]:
import mxnet as mx
model_file_name = 'model.tar.gz'
model_full_path = f'{fm.output_path}/{fm.latest_training_job.job_name}/output/{model_file_name}'
print(f'Model Path: {model_full_path}')

# Factorization Machine 모형 다운로드 
os.system('aws s3 cp ' + model_full_path + ' .')

# MXNet에 로드할 모형 파일 추출
os.system('tar xzvf ' + model_file_name)
os.system('unzip -o model_algo-1')
os.system('mv symbol.json model-symbol.json')
os.system('mv params model-0000.params')

### 모형 데이터를 추출하여 항목 및 사용자 잠재 행렬 생성

이제 Factorization Machine을 훈련한 후 각 사용자 및 항목을 나타내는 값을 추출하겠습니다. 훈련의 결과는 두 개의 행렬이며, 함께 곱하면 목표 값(0 또는 1)을 가능한 한 가깝게 나타냅니다.

보다 수학적 용어로 Factorization Machine 모형 출력은 3개의 N차원 배열(ndarrays)로 구성됩니다.

    V – (N x k) 행렬, 여기서
        k는 잠재 공간의 차원
        N은 사용자 및 항목의 총 수
    w – N차원 벡터
    b – 단일 숫자: 편향 용어

특성으로 사용할 이러한 값을 추출하려면 먼저 모형을 로드해야 합니다. 그런 다음 세 행렬 각각의 값을 추출하고 `knn_item_matrix` 및 t`knn_user_matrix` 행렬을 구축합니다.

In [ ]:
# 모형 데이터 추출
m = mx.module.Module.load('./model', 0, False, label_names=['out_label'])
V = m._arg_params['v'].asnumpy()
w = m._arg_params['w1_weight'].asnumpy()
b = m._arg_params['w0_weight'].asnumpy()

nb_users = customers.shape[0]
nb_item = products.shape[0]

# 항목 잠재 행렬 - concat(V[i], w[i]).  
knn_item_matrix = np.concatenate((V[nb_users:], w[nb_users:]), axis=1)
knn_train_label = np.arange(1,nb_item+1)

# 사용자 잠재 행렬 - concat (V[u], 1) 
ones = np.ones(nb_users).reshape((nb_users, 1))
knn_user_matrix = np.concatenate((V[:nb_users], ones), axis=1)

## KNN(k-최근접 이웃) 모형 구축

이제 훈련 데이터가 확보되었으므 KNN 모형에 이를 제공할 수 있습니다. 이전과 마찬가지로 protobuf IO 형식 데이터를 Amazon S3에 저장하고, 모형을 인스턴스화하고, 하이퍼파라미터를 설정해야 합니다.

먼저 경로 및 추정기를 설정합니다.

In [ ]:
print('KNN train features shape = ', knn_item_matrix.shape)
knn_prefix = 'knn'
train_key = 'train_knn'
knn_output_prefix = f's3://{bucket}/{knn_prefix}/output'
knn_train_data_path = writeDatasetToProtobuf(knn_item_matrix, bucket, 
                                             knn_prefix, train_key, 
                                             "dense", 
                                             knn_train_label)
print(f'Uploaded KNN train data: {knn_train_data_path}')

nb_recommendations = 100

# 추정기 설정
knn = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, "knn"),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=knn_output_prefix,
    sagemaker_session=sagemaker.Session()
)

이제 하이퍼파라미터를 설정합니다. 이 접근 방식에서는 KNN(k-최근접 이웃)에 기본 `index_type` 파라미터를 사용합니다. 이 파라미터는 정확하지만 대규모 데이터 집합에서는 속도가 느려질 수 있습니다. 이 경우 근사치지만 더 빠른 답을 얻을 수 있는 다른 '`index_type` 파라미터를 사용하는 것이 좋습니다.

인덱스 유형에 대한 자세한 내용은 [k-NN 하이퍼파라미터](https://docs.aws.amazon.com/sagemaker/latest/dg/kNN_hyperparameters.html) 페이지를 참조하십시오.

In [ ]:
knn.set_hyperparameters(feature_dim=knn_item_matrix.shape[1], 
                        k=nb_recommendations, 
                        index_metric="INNER_PRODUCT", 
                        predictor_type='classifier', 
                        sample_size=200000)


knn.fit({'train': knn_train_data_path})

이제 훈련된 모형이 있으므로 배치 추론에 참조할 수 있도록 이를 저장합니다.

In [ ]:
knn_model_name = knn.latest_training_job.job_name
print("created model: ", knn_model_name)

# 배치 추론 중 다음 단계에서 참조할 수 있도록 모형 저장
sm = boto3.client(service_name='sagemaker')
primary_container = {
    'Image': knn.image_name,
    'ModelDataUrl': knn.model_data,
}

knn_model = sm.create_model(
        ModelName = knn.latest_training_job.job_name,
        ExecutionRoleArn = knn.role,
        PrimaryContainer = primary_container)
print("saved the model")

## 배치 변환

모형이 수행한 예측을 보려면 추론을 생성하고 해당 추론이 적합한지 확인해야 합니다. 지난 번처럼 이 프로세스를 반복하고 가능한 모든 항목 조합으로 한 번에 한 명의 사용자를 확인할 수도 있습니다. 하지만 Amazon SageMaker에서는 전체 데이터 집합에 대한 추론을 수행하는 데 사용할 수 있는 배치 변환 작업을 제공합니다. 자세한 내용은 [배치 변환을 사용하여 전체 데이터 세트에 대한 추론 가져오기](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html) 섹션을 참조하십시오.

이 섹션에서는 배치 변환을 사용하여 모든 사용자를 위한 상위 100개의 추천을 예측합니다.

In [ ]:
# 추론 데이터를 S3에 업로드
knn_batch_data_path = writeDatasetToProtobuf(knn_user_matrix,
                                             bucket, 
                                             knn_prefix, 
                                             train_key, 
                                             "dense")
print ("Batch inference data path: ",knn_batch_data_path)

# transformer 객체 초기화
transformer =sagemaker.transformer.Transformer(
    base_transform_job_name="knn",
    model_name=knn_model_name,
    instance_count=1,
    instance_type=instance_type,
    output_path=knn_output_prefix,
    accept="application/jsonlines; verbose=true",
    
)

# 변환 작업 시작
transformer.transform(knn_batch_data_path, 
                      content_type='application/x-recordio-protobuf',
                      split_type='RecordIO')
transformer.wait()

 

이제 예측을 자유롭게 검사할 수 있습니다. 먼저 예측을 다운로드합니다.

In [ ]:
# 예측 다운로드 
results_file_name = "inference_output"
inference_output_file = "knn/output/train_knn.out"
s3_client = boto3.client('s3')
s3_client.download_file(bucket, inference_output_file, results_file_name)

In [ ]:
# 파일을 열고 메모리에 로드
with open(results_file_name) as f:
    results = f.readlines() 

결과에는 100개의 최근접 이웃 영화 ID와 이에 해당하는 거리가 포함됩니다. 사용자 번호 200의 경우에 어떤 모습인지 확인합니다.

In [ ]:
test_user_idx = 200
u_one_json = json.loads(results[test_user_idx])
recommended_movies = [int(movie_id) for movie_id in u_one_json['labels']]
distances = [round(distance, 4) for distance in u_one_json['distances']]

print(f'Recommended movie Ids for user #{test_user_idx} : {recommended_movies}')

print(f'Movie distances for user #{test_user_idx} : {distances}')

사용자 200의 취향에 가장 가까운 영화들을 확보했습니다. 이제 해당 제목을 볼 수 있습니다.

In [ ]:
titles_200 = reduced_df[reduced_df.item.isin(recommended_movies)].product_title.unique()
titles_200

이를 사용자 200이 좋아하는 영화와 비교합니다.

In [ ]:
reduced_df.query('user==200 &amp; star_rating == 5')

**질문:** 이러한 추천이 일리가 있다고 생각하시나요? 그렇다거나 그렇지 않다고 생각하는 이유를 설명하세요.

In [ ]:
# 여기에 답변 입력:

In [ ]:
np.isin(titles_200, titles.tail(100).product_title.unique()).sum()

**슈퍼 보너스 질문:** 사용자 201에 대한 예측을 복구하여 사용자 200과 비교하여 어떤지 확인합니다. 여전히 서로 상관 관계가 있나요? 이 접근 방식이 첫 번째 regressor보다 개선되었다고 생각하시나요?

In [ ]:
# 사용자 201에 대한 예측 복구

test_user_idx = 201
u_one_json = json.loads(results[test_user_idx])
recommended_movies_201 = [int(movie_id) for movie_id in u_one_json['labels']]

In [ ]:
# 추천 인쇄

titles_201 = reduced_df[reduced_df.item.isin(recommended_movies_201)].product_title.unique()
titles_201

In [ ]:
# 두 예측 비교

overlap = np.isin(titles_200, titles_201).sum()
print(f'The recommendations for "user 201" that are present in "user 200" are: {overlap} out of: {len(titles_200)}')

In [ ]:
# 사용자 201 선호 비디오와 비교

reduced_df.query('user==201 &amp; star_rating == 5')

In [ ]:
test_user_idx = 900
u_one_json = json.loads(results[test_user_idx])
recommended_movies_900 = [int(movie_id) for movie_id in u_one_json['labels']]
titles_900 = reduced_df[reduced_df.item.isin(recommended_movies_201)].product_title.unique()
overlap_900 = np.isin(titles_200, titles_900).sum()
print(f'The recommendations for "user 900" that are present in "user 200" are: {overlap} out of: {len(titles_200)}')
reduced_df.query('user==900 &amp; star_rating == 5')

In [ ]:
# 여기에 답변 입력:

평점 외에 특성 추가, 다양한 특성 선택 시도, 하이퍼파라미터 튜닝, 모형 변경 등 이러한 모형을 개선하기 위해 수행할 수 있는 여러 가지 방법이 있습니다. 가장 정교한 추천 알고리즘은 딥 러닝을 기반으로 합니다. 이 또한 탐색할 수 있습니다.

마무리할 시간입니다! 이제 사용자에게 상위 100개의 영화를 알려줄 수 있는 추천 시스템이 생겼습니다. 자유롭게 하이퍼파라미터와 데이터를 최적화하고 튜닝하여 더 나은 추천 시스템을 만들 수 있는지 확인해 보시기 바랍니다.

### 결론

이 노트북에서는 Amazon SageMaker 기본 제공 알고리즘만 사용하여 다양한 기법을 통해 추천 시스템을 만들었습니다. 다양한 형식으로 데이터를 준비하고 특성 엔지니어링을 수행하는 방법을 배웠습니다. 훈련된 모형의 문제를 식별하고 다양한 방식으로 문제를 재구성하여 최종 결과를 얻을 수 있었습니다. 

이제는 알 수 있겠지만, 모형 훈련에는 많은 단계, 준비 및 검증이 필요합니다. 모형 훈련은 간소화된 프로세스가 아니라 반복적인 프로세스입니다. 일반적으로 다음과 같은 단계로 이루어진 선순환이라고 생각할 수 있습니다.

- (비즈니스) 문제를 정의합니다.
- 비즈니스 문제를 기계 학습 문제로 구성합니다.
- 데이터를 준비하고 특성 엔지니어를 수행합니다.
- 모형을 훈련하고 평가합니다.
- 모형(추론)을 배포합니다.
- 모니터링하고 평가합니다.

모든 단계에는 저다마 어려움이 있으며 각 단계가 서로 영향을 미칩니다. 따라서 모형 훈련만이 아니라 전체 파이프라인에 주의를 기울이는 것이 중요합니다.
